In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.preprocessing import StandardScaler,LabelEncoder
from scipy import stats
from scipy.stats import norm,skew
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer,log_loss
from tqdm.notebook import tqdm_notebook
from sklearn import preprocessing
from sklearn.impute import KNNImputer

#Merge Train/Test Data and Macro Data

In [ ]:
df=pd.read_csv("train.csv", parse_dates=['timestamp'])
df1= pd.read_csv("test.csv", parse_dates=['timestamp'])
df_macro=pd.read_csv("macro.csv", parse_dates=['timestamp'])
train = pd.merge(df,df_macro, how='left', on='timestamp')
test = pd.merge(df1, df_macro, how= 'left', on = 'timestamp')

#Data Cleaning

##0. Concatenating Train and Test Dataset

In [ ]:
train_data = train.drop('price_doc', axis=1)
X = pd.concat([train_data, test])
print(X.shape)

(38133, 390)


##1. Drop some columns
- columns which has correlation value with respect to price is less than 5%
- colums which has almost 0 variance

In [ ]:
corr = train.corr()
cor_target = abs(corr["price_doc"])
relevant_features = cor_target[cor_target<=0.05]
# relevant_features.index.to_list()

In [ ]:
ids = [i for i in X.columns if 'ID' in i.split('_')]
print(ids)

['ID_metro', 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2', 'ID_railroad_terminal', 'ID_bus_terminal']


In [ ]:
useless_ids = ['ID_metro', 'ID_railroad_station_walk', 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2', 'ID_railroad_terminal', 'ID_bus_terminal']
X.drop(list(relevant_features.index) + useless_ids, axis = 1, inplace = True)
X.shape

(38133, 316)

#2. Add useful features

In [ ]:
X['timestamp'] = pd.to_datetime(X['timestamp'])
X['year'] = X['timestamp'].dt.year #Year
X['month'] = X['timestamp'].dt.month #Month
X['day_of_week'] = X['timestamp'].dt.dayofweek # Day of Week
X['rel_floor'] = X['floor'] / X['max_floor'] # Floor relative to total floors in building
X['room_size'] = X['life_sq'] / X['num_room'] # Room size
X['add_sq'] = X['full_sq'] - X['life_sq'] # Non living area
X['add_area_ratio'] = X['add_sq']/X['full_sq'] #Ratio for non living area
X['school_per_area'] = 1e7 * X['school_education_centers_raion'] / X['area_m'] # Schools per area
X['school_vicinity'] = X['school_km'] / X['area_m'].apply(np.sqrt, 0) # Schools in vicinity
X['office_vicinity'] = X['office_km'] / X['area_m'].apply(np.sqrt, 0) # Office vicinity
X['healthcare_vicinity'] = X['public_healthcare_km'] / X['area_m'].apply(np.sqrt, 0) # Public heathcare vicinity
X['mall_vicinity'] =  X['shopping_centers_km'] / X['area_m'].apply(np.sqrt, 0) # Shopping mall vicinity
X['resident_to_total_ratio'] = X['life_sq']/X['full_sq'] # Residential & kitchen area to total area ratio
X['avg_room_area'] = X['life_sq']/X['num_room'] # Average area per room
X['pct_labor_force'] = X['work_all']/X['raion_popul'] # Percentage of population in labor force
X['floor_rel_total'] = X['floor']/X['max_floor'] # Apartment floor relative to building height

#Some additional Binary Variables
X['metro_flag'] = np.where(X['raion_popul'] > 150000, 1, 0)
X['large_flag'] = np.where(X['max_floor'] >= 20, 1, 0)
X['small_flag'] = np.where(X['max_floor'] <= 20, 1, 0)

# Average building height for subarea
sub_area_means = X.groupby('sub_area').agg({'max_floor':np.mean}).reset_index().rename(columns={'max_floor':'mean_bldg_height'})
X = pd.merge(X, sub_area_means, on = ['sub_area'], how = 'left')

# Average distance to Kremlin by subarea
dist_to_kremlin = X.groupby('sub_area').agg({'kremlin_km':np.nanmean}).reset_index().rename(columns={'kremlin_km':'subarea_dist_to_kremlin'})
X = pd.merge(X, dist_to_kremlin, on = ['sub_area'], how = 'left')

#3. Data Anomalies

Cleaning data by correcting data anomalies based on the following assumptions:

- Total square area(full_sq) and living area(life_sq) should fall within a certain range
- Living area(life_sq) should not be greater than total square area(full_sq)
- Kitchen area(kitch_sq) should not be greater than living area(life_sq)
- Ratio of living area to total area
- Build year(build_year) should be within a certain range
- Number of living rooms(num_room) should not be 0
- Floor of the building(floor) and number of floors in the building(max_floor) should not be 0
- Floor of the building(floor) should not be greater than number of floors in the building(max_floor)
- Price(price_doc) per total square area(full_sq) should be within a certain range
- Return Index of each room is less then 5  and replace it as NaN


In [ ]:
#Return Index of life sq < 7 and replace it as NaN
replace_index = X[X.life_sq < 7].index
X.loc[replace_index, 'life_sq'] = np.NaN

#Return Index of full sq < 7 and replace it as NaN
replace_index = X[X.life_sq < 7].index
X.loc[replace_index, 'full_sq'] = np.NaN

#If life > full NaN
replace_index = X[X.life_sq > X.full_sq].index
X.loc[replace_index, 'life_sq'] = np.NaN

replace_index = X[(X.full_sq > 200) & (X.life_sq/X.full_sq < 0.3)].index
X.loc[replace_index, 'full_sq'] = np.NaN

replace_index = X[X.num_room == 0].index
X.loc[replace_index, 'num_room'] = np.NaN

replace_index = X[X.floor == 0].index
X.loc[replace_index, 'floor'] = np.NaN

replace_index = X[X.max_floor == 0].index
X.loc[replace_index, 'max_floor'] = np.NaN

replace_index = X[X.floor > X.max_floor].index
X.loc[replace_index, 'max_floor'] = np.NaN

#Return Index of each room is less then 5  and replace it as NaN
replace_index  =  X[X.life_sq/X.num_room  < 5].index
X.loc[replace_index, "num_room"] = np.NaN

#Filling Missing data for life square
X['life_sq_percentage'] = X.life_sq.div(X.full_sq)
X.life_sq=X.life_sq.fillna(X.life_sq_percentage.mean()*X.full_sq)

#4. Missing Data Analysis

In [ ]:
missing_values = ((X.isna().sum())/X.shape[0])*100

missing_values.sort_values(ascending=False,inplace = True)
missing_values.head(50)

provision_retail_space_modern_sqm        98.025332
museum_visitis_per_100_cap               64.414025
theaters_viewers_per_1000_cap            64.414025
hospital_beds_raion                      46.833451
room_size                                39.047544
avg_room_area                            39.047544
state                                    37.377075
max_floor                                30.713555
population_reg_sports_share              28.586788
share_own_revenues                       28.586788
profitable_enterpr_share                 28.586788
unprofitable_enterpr_share               28.586788
hospital_bed_occupancy_per_year          28.586788
hospital_beds_available_per_cap          28.586788
power_clinics                            28.586788
provision_doctors                        28.586788
fin_res_per_cap                          28.586788
apartment_build                          28.586788
marriages_per_1000_cap                   28.586788
construction_value             

##1) Drop all the colums having missing values more than 50%

In [ ]:
X = X.drop(columns = missing_values[(missing_values>50)].index.to_list())

##2) Median Imputation for Columns which have missing values less than 10%

In [ ]:
# Select all the columns having missing values less than equal to 10%

print(len(missing_values[(missing_values>0) & (missing_values<=10)].index))
missing_values[(missing_values>0) & (missing_values<=10)].index.to_list()

38


['cafe_avg_price_2000',
 'cafe_sum_2000_max_price_avg',
 'cafe_sum_2000_min_price_avg',
 'deposits_rate',
 'oil_urals',
 'micex_cbi_tr',
 'gdp_quart',
 'gdp_deflator',
 'gdp_quart_growth',
 'cpi',
 'ppi',
 'average_provision_of_build_contract_moscow',
 'balance_trade_growth',
 'usdrub',
 'average_provision_of_build_contract',
 'micex_rgbi_tr',
 'rts',
 'eurrub',
 'brent',
 'gdp_annual',
 'gdp_annual_growth',
 'deposits_value',
 'fixed_basket',
 'mortgage_growth',
 'rent_price_4+room_bus',
 'mortgage_rate',
 'rent_price_2room_bus',
 'rent_price_3room_bus',
 'rent_price_3room_eco',
 'prom_part_5000',
 'floor',
 'metro_min_walk',
 'metro_km_walk',
 'railroad_station_walk_km',
 'railroad_station_walk_min',
 'product_type',
 'green_part_2000',
 'full_sq']

In [ ]:
values = {'cafe_sum_2000_max_price_avg': X['cafe_sum_2000_max_price_avg'].median(),
          'cafe_sum_2000_min_price_avg' : X['cafe_sum_2000_min_price_avg'].median(),
          'cafe_avg_price_2000': X['cafe_avg_price_2000'].median(),
          'deposits_rate': X['deposits_rate'].median(),
          'deposits_value': X['deposits_value'].median(),
          'fixed_basket':X['fixed_basket'].median(),
          'micex_cbi_tr': X['micex_cbi_tr'].median(),
          'micex_rgbi_tr': X['micex_rgbi_tr'].median(),
          'rts': X['rts'].median(),
          'mortgage_growth': X['mortgage_growth'].median(),
           'average_provision_of_build_contract_moscow': X['average_provision_of_build_contract_moscow'].median(),
           'brent': X['brent'].median(),
           'mortgage_rate': X['mortgage_rate'].median(),
           'eurrub': X['eurrub'].median(),
           'usdrub': X['usdrub'].median(),
           'balance_trade_growth': X['balance_trade_growth'].median(),
           'gdp_deflator': X['gdp_deflator'].median(),
           'average_provision_of_build_contract': X['average_provision_of_build_contract'].median(),
           'ppi': X['ppi'].median(),
           'cpi': X['cpi'].median(),
           'gdp_quart_growth': X['gdp_quart_growth'].median(),
           'gdp_quart': X['gdp_quart'].median(),
           'gdp_annual': X['gdp_annual'].median(),
           'oil_urals': X['oil_urals'].median(),
           'rent_price_3room_eco': X['rent_price_3room_eco'].median(),
           'rent_price_2room_bus': X['rent_price_2room_bus'].median(),
           'rent_price_3room_bus': X['rent_price_3room_bus'].median(),
           'rent_price_4+room_bus': X['rent_price_4+room_bus'].median(),
           'gdp_annual_growth': X['gdp_annual_growth'].median(),
           'prom_part_5000': X['prom_part_5000'].median(),
           'floor': X['floor'].median(),
           'metro_min_walk': X['metro_min_walk'].median(),
           'metro_km_walk': X['metro_km_walk'].median(),
           'railroad_station_walk_km': X['railroad_station_walk_km'].median(),
           'railroad_station_walk_min': X['railroad_station_walk_min'].median(),
           'product_type': X['product_type'].mode(),
            'green_part_2000': X['green_part_2000'].median(),
           'full_sq': X['full_sq'].median()
           }

X.fillna(value=values, inplace = True)

##2. Using K-nearest Model for others

In [ ]:
missing_values[(missing_values>10) & (missing_values<=50)].index

Index(['hospital_beds_raion', 'room_size', 'avg_room_area', 'state',
       'max_floor', 'population_reg_sports_share', 'share_own_revenues',
       'profitable_enterpr_share', 'unprofitable_enterpr_share',
       'hospital_bed_occupancy_per_year', 'hospital_beds_available_per_cap',
       'power_clinics', 'provision_doctors', 'fin_res_per_cap',
       'apartment_build', 'marriages_per_1000_cap', 'construction_value',
       'child_on_acc_pre_school', 'invest_fixed_assets_phys', 'pop_migration',
       'pop_total_inc', 'incidence_population', 'perinatal_mort_per_1000_cap',
       'infant_mortarity_per_1000_cap', 'electric_stove_share', 'gas_share',
       'housing_fund_sqm', 'grp', 'real_dispos_income_per_cap_growth',
       'num_room', 'rel_floor', 'floor_rel_total', 'modern_education_share',
       'old_education_build_share', 'material', 'life_sq_percentage',
       'preschool_quota', 'cafe_avg_price_1000', 'cafe_sum_1000_min_price_avg',
       'cafe_sum_1000_max_price_avg', 'reside

In [ ]:
missing_values_list = ['hospital_beds_raion', 'room_size', 'avg_room_area', 'state',
       'max_floor', 'population_reg_sports_share', 'share_own_revenues',
       'profitable_enterpr_share', 'unprofitable_enterpr_share',
       'hospital_bed_occupancy_per_year', 'hospital_beds_available_per_cap',
       'power_clinics', 'provision_doctors', 'fin_res_per_cap',
       'apartment_build', 'marriages_per_1000_cap', 'construction_value',
        'invest_fixed_assets_phys', 'pop_migration',
       'pop_total_inc', 'incidence_population', 'perinatal_mort_per_1000_cap',
       'infant_mortarity_per_1000_cap', 'electric_stove_share', 'gas_share',
       'housing_fund_sqm', 'grp', 'real_dispos_income_per_cap_growth',
       'num_room', 'rel_floor', 'floor_rel_total', 'material', 'life_sq_percentage',
       'preschool_quota', 'cafe_avg_price_1000', 'cafe_sum_1000_min_price_avg',
       'cafe_sum_1000_max_price_avg', 'resident_to_total_ratio',
       'add_area_ratio', 'add_sq', 'raion_build_count_with_material_info',
       'build_count_1946-1970', 'build_count_before_1920',
       'raion_build_count_with_builddate_info', 'build_count_monolith',
       'build_count_brick', 'cafe_sum_1500_min_price_avg',
       'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500',
       'retail_trade_turnover_per_cap', 'unemployment', 'childbirth', 'salary',
       'invest_fixed_assets', 'retail_trade_turnover_growth', 'salary_growth',
       'retail_trade_turnover', 'pop_natural_increase', 'labor_force',
       'invest_fixed_capital_per_cap', 'employment', 'students_state_oneshift',
       'seats_theather_rfmin_per_100000_cap', 'provision_nurse',
       'bandwidth_sports', 'turnover_catering_per_cap', 'load_on_doctors',
       'load_of_teachers_school_per_teacher', 'average_life_exp']

In [ ]:
X.replace([np.inf, -np.inf], np.NaN, inplace=True)

imputer = KNNImputer(n_neighbors=3)
for i in missing_values_list:
    X[i] = imputer.fit_transform(X[[i]])

In [ ]:
print(X.shape)

(38133, 335)


In [ ]:
X = X.drop(columns = ['child_on_acc_pre_school', 'old_education_build_share', 'modern_education_share'])

In [ ]:
X.product_type = X.product_type.fillna('Investment')

In [ ]:
missing_val1 = X.isna().sum()
missing_val_per = X.isna().sum()/len(train)*100
missing_val_table = pd.concat([missing_val1, missing_val_per], axis=1)
missing_val_table_columns = missing_val_table.rename(columns = {0: 'Missing Values', 1: '% of Total Values'})
missing_val_table_columns = missing_val_table_columns[missing_val_table_columns.iloc[:,:] != 0].sort_values('% of Total Values', ascending=False).round(1)
missing_val_table_columns = missing_val_table_columns[missing_val_table_columns['Missing Values'].notna()]
missing_val_table_columns

,Missing Values,% of Total Values


In [ ]:
train_price = train[['id','price_doc']].copy()

In [ ]:
data_train = X[X['id'].isin(train_price['id'])]
data_train = pd.merge(data_train, train_price, on = ['id'], how = 'inner')
print(data_train.shape)

data_test = X[~X['id'].isin(train_price['id'])]
print(data_test.shape)

(30471, 333)
(7662, 332)


In [ ]:
data_train.to_csv('data_train.csv')
data_test.to_csv('data_test.csv')